# Rotate IP address with Bright Data

When you need to use rotating IP to scrape the data from a website:
- You wish to keep it secret
- to avoid getting blocked when you're scra
- to avoid API rate limit (and hit 429 error) when you're scraping too much with same IP address

Warning: this is not 100% success rate.

In [ ]:
%pip install python-dotenv

In [3]:
import os
import random

import requests
from dotenv import load_dotenv

_ = load_dotenv()

url = "https://httpbin.co/ip"

proxy_hostname = os.getenv("PROXY_HOSTNAME")
proxy_username = os.getenv("PROXY_USERNAME")
proxy_password = os.getenv("PROXY_PASSWORD")

In [9]:
class ProxyServer:
    def __init__(self, proxy_username, proxy_password, proxy_hostname):
        self.proxy_username = proxy_username
        self.proxy_password = proxy_password
        self.proxy_hostname = proxy_hostname

    def __call__(self):
        rand_num = random.randint(1, 9999)
        # Reference: https://docs.brightdata.com/api-reference/proxy/rotate_ips
        ## Add -session parameter to this proxy to change IP address every requests is sent
        proxy = f"{self.proxy_username}-session-rand{rand_num}:{self.proxy_password}@{self.proxy_hostname}"
        return {
            "http": f"https://{proxy}",
            "https": f"https://{proxy}",
        }

proxy_servers = ProxyServer(proxy_username, proxy_password, proxy_hostname)
# call proxy_servers() to register a new IP proxy
# >> proxies = proxy_servers() 

In [8]:
# build a session object to preserve the session cookies (or login status, if applicable)
## throughout the process of sending web requests
session = requests.Session()
 
for idx in range(1, 11):
    response = session.request('GET', url=url, 
                proxies=proxy_servers()) # changing IP address every new requests
    print(response.json())

{'method': 'GET', 'ip': '109.198.42.2', 'country': 'TR', 'timezone': 'Europe/Istanbul', 'continent': 'AS'}
